<a href="https://colab.research.google.com/github/cezarviana/fake-news-no-more/blob/main/fake_news_no_more_(verificador_de_noticias).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip -q install google-genai


# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')


# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"


# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk


from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")


# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response


# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é ser um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca google (google_search) para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Priorize fontes jornalísticas reconhecidas e com boa reputação.
        Selecione no máximo 5 lançamentos que demonstrem relevância (baseada na cobertura e qualidade da fonte) e sejam os mais atuais possíveis.
        Se o tópico não possuir 5 notícias a seu respeito, apresente somente as encontradas. Sem adicionar outras notícias com termos parecidos e que não tenham relação direta com o tópico.
        Se o tópico gerar pouca cobertura noticiosa ou reações limitadas, sinalize essa baixa relevância como um possível indicativo para considerar outros tópicos.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca de notícias no Google Search sobre o tópico",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados


################################################
# --- Agente 2: Verificador de Fontes --- #
################################################
def agente_verificador_fontes(topico, lancamentos_buscados):
    verificador_fontes = Agent(
        name="agente_verificador_fontes",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Verificador de Fontes #################################################
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é ser um verificador de fontes, especialista em fact-checking.
        Para cada fonte principal identificada:
        - Determine se o site ou canal é conhecido e geralmente considerado confiável.
        - Localize e examine a seção 'Sobre nós' (ou equivalente) para entender a missão, equipe e possíveis vieses do site.
        - Verifique se outras fontes confiáveis corroboram as informações apresentadas pela fonte principal. Liste as fontes que confirmam os achados.
        """,
        description="Agente que verifica as fontes levantadas",
        tools=[google_search]
    )

    entrada_do_agente_verificador_fontes = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    verificacao_fontes = call_agent(verificador_fontes, entrada_do_agente_verificador_fontes)
    return verificacao_fontes


################################################
# --- Agente 3: Verificador de Conteúdo --- #
################################################
def agente_verificador_conteudo(topico, lancamentos_buscados):
    verificador_conteudo = Agent(
        name="agente_verificador_conteudo",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Verificador de Conteúdo #################################################
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é ser um verificador de conteúdo, especialista em fact-checking.
        Examine o conteúdo das notícias e informações relacionadas ao tópico.
        Linguagem e Estilo:
        - Avalie se há uso de sofismos ou outras técnicas de persuasão manipuladoras.
        - Identifique erros de ortografia e gramática que possam indicar falta de profissionalismo ou revisão.
        - Distinga claramente entre fatos apresentados e opiniões, verificando se as opiniões são devidamente atribuídas.
        Contexto e Evidências:
        - Verifique se a data da informação é relevante para o contexto atual.
        - Se houver imagens, utilize a busca reversa (Google Imagens) para verificar sua autenticidade e se foram usadas em outros contextos enganosos.
        Coerência:
        - Avalie a lógica interna do conteúdo e sua coerência com informações de outras fontes.
        """,
        description="Agente que verifica o conteúdo das notícias levantadas",
        tools=[google_search]
    )

    entrada_do_agente_verificador_conteudo = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    verificacao_conteudo = call_agent(verificador_conteudo, entrada_do_agente_verificador_conteudo)
    return verificacao_conteudo


##########################################
# --- Agente 4: Agência de Fact-Checking --- #
##########################################
def agente_verificador_fatos(topico, data_de_hoje):
    verificador_fatos = Agent(
        name="agente_verificador_fatos",
        model="gemini-2.0-flash",
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua tarefa é verificar se o tópico/afirmação já foi alvo de checagem por agências de fact-checking confiáveis, através da  busca google (google_search), priorizando as verificações mais recentes sobre o tópico.
        As agências de fact-checking a serem consultadas de acordo com o tópico serão as indicadas abaixo:
        - No Brasil:
          - Aos Fatos;
          - Lupa;
          - UOL Confere;
          - Estadão Verifica;
          - Fato ou Fake (G1);
          - Boatos.org;
          - Agência Pública - Truco no Congresso;
          - Comprova;
          - E-farsas;
          - É isso Mesmo? (O Globo);
          - Portal EBC - Checagem;
        - Internacionais com atuação ou relevância no Brasil:
          - AFP Fact Check;
          - Reuters Fact Check;
          - Snopes;
          - PolitiFact;
          - FactCheck.org;
        - Organizações e Redes Internacionais:
          - International Fact-Checking Network (IFCN);
          - European Fact-Checking Standards Network (EFCSN);
          - Duke Reporters' Lab;
        Informe se o tópico foi encontrado em alguma das agências de fact-checking e qual foi a conclusão dessas agências (verdadeiro, falso, enganoso, etc.).
        Se encontrado, cite a fonte da agência de fact-checking e um breve resumo da sua análise.
        """,
        description="Agente que verifica o que agências de fact-checking dizem a respeito do tópico",
        tools=[google_search]
    )

    entrada_do_agente_verificador_fatos = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    verificacao_fatos = call_agent(verificador_fatos, entrada_do_agente_verificador_fatos)
    return verificacao_fatos


###############################################
# --- Agente 5: Organizador do Resultado da Verificação --- #
################################################
def agente_organizador_resultado(topico, lancamentos_buscados, verificacao_fontes, verificacao_conteudo, verificacao_fatos):
    organizador = Agent(
        name="agente_organizador_resultado",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente de Resultados #################################################
        instruction="""
        Você é parte de um sistema colaborativo de verificação de notícias. Siga rigorosamente as instruções específicas da sua função para analisar o tópico fornecido e contribuir para a determinação da sua veracidade.
        Sua função é organizar os resultados com base nas análises dos outros agentes sobre o tópico, e determinar a veracidade da informação.
        Apresente um veredicto claro: Verdadeiro, Falso, Enganoso, Insustentável, etc.
        Justifique sua conclusão de forma concisa, utilizando as evidências e os resultados fornecidos pelos outros agentes (agente_buscador, agente_verificador_fontes, agente_verificador_conteudo e agente_verificador_fatos).
        Liste as fontes mais relevantes (sites de notícias confiáveis, agências de fact-checking) que sustentam sua conclusão.
        """,
        description="Agente que organiza os resultados da verificação",
        tools=[google_search]
    )

    entrada_do_agente_organizador_resultado = f"Tópico:{topico}\nLançamentos buscados:{lancamentos_buscados}\nVerificação fontes:{verificacao_fontes}\nVerificação conteúdo:{verificacao_conteudo}\nVerificação fatos:{verificacao_fatos}"
    # Executa o agente
    resultado = call_agent(organizador,  entrada_do_agente_organizador_resultado)
    return resultado

In [4]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Verificação de Fatos com 5 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer saber a veracidade: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos pesquisar sobre a veracidade a respeito de {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador de Notícias) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    verificacao_fontes = agente_verificador_fontes(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Verificador de Fontes) ---\n")
    display(to_markdown(verificacao_fontes))
    print("--------------------------------------------------------------")

    verificacao_conteudo = agente_verificador_conteudo(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 3 (Verificador de Conteúdo) ---\n")
    display(to_markdown(verificacao_conteudo))
    print("--------------------------------------------------------------")

    verificacao_fatos = agente_verificador_fatos(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 4 (Agências de Fact-Checking) ---\n")
    display(to_markdown(verificacao_fatos))
    print("--------------------------------------------------------------")

    resultado = agente_organizador_resultado(topico, lancamentos_buscados, verificacao_fontes, verificacao_conteudo, verificacao_fatos)
    print("\n--- 📝 Resultado do Agente 5 (Organizador do Resultado da Verificação) ---\n")
    display(to_markdown(resultado))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Verificação de Fatos com 5 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer saber a veracidade: Tarifas dos EUA vão favorecer a sua economia?
Maravilha! Vamos pesquisar sobre a veracidade a respeito de Tarifas dos EUA vão favorecer a sua economia?

--- 📝 Resultado do Agente 1 (Buscador de Notícias) ---



> Okay, vou pesquisar no Google Search por notícias recentes e relevantes sobre o impacto das tarifas dos EUA na economia americana.
> 
> 
> Com base nas notícias mais recentes, as tarifas dos EUA parecem estar tendo um impacto misto e, em alguns casos, negativo, na economia americana:
> 
> 1.  **Desaceleração Econômica:** A economia dos EUA mostrou sinais de desaceleração antes mesmo do impacto total das tarifas de Trump ser sentido. No início de 2025, a economia perdeu fôlego com consumidores mais cautelosos e um aumento do déficit comercial devido à corrida para importar produtos antes das tarifas.
> 2.  **Impacto no PIB:** A estimativa inicial do governo dos EUA para o PIB do primeiro trimestre de 2025 indicou um crescimento a uma taxa anualizada de apenas 0,4%, o ritmo mais fraco em quase três anos. Alguns economistas reduziram drasticamente suas estimativas do PIB devido a um aumento no déficit do comércio de mercadorias. Uma notícia da CNN Brasil relatou uma contração no primeiro trimestre, prejudicada por importações antecipadas para evitar tarifas mais altas.
> 3.  **Cautela Empresarial:** Empresas têm se mostrado mais cautelosas em relação a novos gastos, aguardando clareza sobre tarifas, acordos comerciais e política tributária.
> 4.  **Risco de Recessão:** O FMI aumentou o risco de recessão para a economia dos EUA de 25% para 40% devido à guerra comercial. O FMI também estima que o crescimento nos Estados Unidos diminua para 1,8%, inferior ao projetado anteriormente, devido à incerteza política e tensões comerciais.
> 5.  **Tensões Comerciais:** As tarifas levaram a retaliações de outros países, como a China, aumentando as tensões no comércio internacional.
> 
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Verificador de Fontes) ---



> Okay, com base nas informações fornecidas, vou verificar as fontes que estão sendo usadas para embasar as afirmações sobre o impacto das tarifas dos EUA na economia americana. As principais alegações são:
> 
> *   Desaceleração econômica dos EUA antes do impacto total das tarifas.
> *   Impacto negativo no PIB do primeiro trimestre de 2025.
> *   Cautela empresarial devido à incerteza sobre tarifas e políticas.
> *   Aumento do risco de recessão, conforme avaliação do FMI.
> *   Tensões comerciais internacionais devido a retaliações tarifárias.
> 
> Para verificar essas alegações, focarei em identificar as fontes originais das notícias e relatórios, avaliar sua credibilidade e buscar confirmação em outras fontes confiáveis.
> 
> **Próximos passos:**
> 
> 1.  **Identificar as fontes primárias:** Buscar as notícias e relatórios originais que mencionam a desaceleração econômica, o impacto no PIB, a cautela empresarial, a avaliação do FMI e as tensões comerciais.
> 2.  **Analisar a credibilidade das fontes:** Avaliar a reputação dos veículos de notícias, a expertise dos economistas citados e a metodologia dos relatórios.
> 3.  **Buscar confirmação em outras fontes:** Verificar se outras fontes de notícias, instituições financeiras e especialistas confirmam as alegações e os dados apresentados.
> 
> Com base nos resultados da pesquisa, aqui está uma análise das fontes e das alegações sobre o impacto das tarifas dos EUA:
> 
> **1. Desaceleração Econômica e Impacto no PIB:**
> 
> *   **Fontes:** CNN Brasil, Money Times, SAPO, Forbes.
> *   **Análise:** Várias fontes de notícias, incluindo a CNN Brasil e Money Times, relatam uma desaceleração na economia dos EUA em 2025, com algumas indicando uma contração no primeiro trimestre. A CNN Brasil atribui essa contração, em parte, ao aumento das importações pelas empresas que buscavam evitar as tarifas de Trump.
> *   **Credibilidade:** A CNN Brasil é uma fonte de notícias amplamente reconhecida. O Money Times é um portal de notícias de economia e finanças.
> *   **Confirmação:** A desaceleração econômica e o impacto no PIB são confirmados por outras fontes, como a Forbes, que cita economistas da Reuters.
> 
> **2. Cautela Empresarial:**
> 
> *   **Fontes:** CNN Brasil, Forbes.
> *   **Análise:** As notícias indicam que as empresas estão mais cautelosas em relação a novos gastos devido à incerteza em torno das tarifas, acordos comerciais e políticas tributárias.
> *   **Credibilidade:** As fontes são geralmente confiáveis, como mencionado acima.
> *   **Confirmação:** A Forbes corrobora essa alegação, mencionando que economistas consultados concordam que as tarifas prejudicaram a confiança das empresas.
> 
> **3. Risco de Recessão (FMI):**
> 
> *   **Fontes:** CNN Brasil, SAPO, BB InvesTalk, ISTOÉ DINHEIRO.
> *   **Análise:** O FMI (Fundo Monetário Internacional) aumentou o risco de recessão para a economia dos EUA devido à guerra comercial. O FMI também reduziu suas projeções de crescimento para os EUA.
> *   **Credibilidade:** O FMI é uma instituição financeira internacional respeitada.
> *   **Confirmação:** Várias fontes de notícias, incluindo CNN Brasil, SAPO, BB InvesTalk e ISTOÉ DINHEIRO, relatam a avaliação do FMI.
> 
> **4. Tensões Comerciais:**
> 
> *   **Fontes:** CNN Brasil, UFF, Me Explica.
> *   **Análise:** As tarifas levaram a retaliações de outros países, como a China, aumentando as tensões no comércio internacional.
> *   **Credibilidade:** As fontes são geralmente confiáveis, como mencionado acima.
> *   **Confirmação:** A UFF e o portal Me Explica também relatam as tensões comerciais e as retaliações tarifárias.
> 
> **Conclusão:**
> 
> Com base nas fontes verificadas, as tarifas dos EUA parecem estar tendo um impacto misto na economia americana. Há evidências de desaceleração econômica, cautela empresarial e aumento do risco de recessão, conforme avaliado pelo FMI. As tarifas também levaram a tensões comerciais internacionais. Embora algumas fontes sugiram que as tarifas visam proteger a indústria americana e reduzir os déficits comerciais, outras indicam que os efeitos negativos superam os benefícios.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Verificador de Conteúdo) ---



...

--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Agências de Fact-Checking) ---



> A análise sobre se as tarifas dos EUA favorecerão ou não a sua economia é complexa e multifacetada, com diferentes especialistas e instituições apresentando visões diversas.
> 
> **Visão Geral dos Impactos:**
> 
> *   **Impacto Negativo no Curto Prazo:** Alguns economistas apontam que, no curto prazo, as tarifas podem ter um impacto negativo na economia dos EUA. Isso ocorre porque as empresas americanas podem enfrentar custos mais altos devido a tarifas sobre matérias-primas e componentes importados, o que pode prejudicar sua competitividade. Além disso, as tarifas podem levar a um aumento dos preços para os consumidores americanos.
> *   **Potencial para Impacto Negativo:** Kellie Meiman Hock, conselheira sênior da McLarty Associates, avalia que o impacto econômico das tarifas impostas pelo governo americano está apenas começando e tende a se intensificar, com os consumidores americanos já sentindo os efeitos das medidas recíprocas.
> *   **Impactos Complexos e Incertos:** A ata de uma reunião do Banco Central destaca que o Brasil parece menos afetado pelas novas tarifas comerciais dos Estados Unidos do que outros países, mas que o país ainda sente o impacto da medida em função do cenário global adverso. O choque de tarifas e o choque de incerteza, apesar de todas as tentativas de mensuração, ainda são de impacto bastante incerto.
> *   **Desaceleração Econômica:** A economia dos EUA já mostra sinais de desaceleração antes mesmo do impacto total das tarifas de Trump, refletindo a cautela dos consumidores e incertezas comerciais.
> *   **Oportunidades para Outros Países:** Alguns especialistas argumentam que a crise gerada pelas tarifas pode criar oportunidades para outros países, como o Brasil, expandirem suas exportações.
> 
> **Visões de Agências de Fact-Checking e Outras Fontes:**
> 
> *   **Estadão Verifica:** Segundo Schwartsman, as tarifas de Trump são um instrumento errado para um problema que não existe, o que pode levar a uma perda de competitividade para a economia americana. Ele também aponta que a confiança na economia americana tomou um golpe severo.
> *   **CNN Brasil:** Fernando Siqueira, da Eleven Financial, comenta que o impacto das tarifas nos Estados Unidos tem sido negativo, principalmente no curto prazo, com um impacto negativo no PIB devido ao aumento das importações em antecipação às tarifas.
> *   **UFMG:** Um estudo da UFMG aponta que a guerra tarifária pode resultar em uma queda de 0,16% no PIB global, com os Estados Unidos enfrentando uma diminuição de US$92,7 bilhões (0,48%) em seu PIB.
> *   **Outras fontes:** Outros especialistas e institutos de pesquisa econômica alertam que, embora as tarifas de Trump possam estimular a produção interna, os efeitos no curto prazo devem ser aumento de preços e menor crescimento econômico.
> 
> **Outras Considerações:**
> 
> *   **Guerra Comercial:** A imposição de tarifas pode levar a retaliações por parte de outros países, resultando em uma guerra comercial que prejudica as cadeias de suprimentos globais e reduz a previsibilidade dos mercados.
> *   **Risco de Recessão:** Com o aumento das tarifas, grandes bancos estão revisando suas projeções e elevando o risco de uma recessão nos Estados Unidos.
> *   **Impacto Setorial:** Embora o impacto geral na economia brasileira possa ser moderado, alguns setores, como o agronegócio, o alumínio e o aço, podem ser significativamente afetados.
> *   **Efeitos a Longo Prazo:** O impacto a longo prazo das políticas tarifárias ainda é incerto, mas tarifas retaliatórias por países afetados já complicaram o comércio global. Altos níveis de dívida também podem corroer a confiança dos investidores, levando a maiores custos de empréstimos e redução do crescimento econômico.
> *   **Negociações e Acordos:** As negociações futuras entre os EUA e outros países serão cruciais para determinar o rumo das relações comerciais e o impacto na economia global.
> 
> Em resumo, a questão de se as tarifas dos EUA favorecerão ou não a sua economia é complexa e depende de uma variedade de fatores, incluindo a magnitude das tarifas, as respostas de outros países, e a capacidade das empresas americanas de se adaptarem às novas condições. Embora alguns argumentem que as tarifas podem proteger empregos e fortalecer as indústrias nacionais, outros alertam para os riscos de aumento de preços, retaliação comercial e desaceleração econômica.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 5 (Organizador do Resultado da Verificação) ---



> Com base nas análises dos agentes, a alegação de que as tarifas dos EUA favorecerão a sua economia é **Insustentável**.
> 
> **Justificativa:**
> 
> *   **Impacto Misto e Predominantemente Negativo:** As tarifas parecem ter um impacto misto, mas predominantemente negativo, na economia dos EUA. Evidências apontam para uma desaceleração econômica, cautela empresarial e aumento do risco de recessão.
> *   **Fontes Confiáveis:** As informações são sustentadas por fontes de notícias confiáveis como CNN Brasil, Forbes e instituições financeiras como o FMI.
> *   **Tensões Comerciais:** As tarifas levaram a retaliações de outros países, aumentando as tensões no comércio internacional.
> *   **Visões Diversas:** Especialistas e instituições apresentam visões diversas, mas a maioria indica que os efeitos negativos superam os benefícios no curto prazo.
> *   **Risco de Recessão:** O FMI aumentou o risco de recessão para a economia dos EUA devido à guerra comercial.
> *   **Impacto no PIB:** Há relatos de impacto negativo no PIB do primeiro trimestre de 2025 devido a importações antecipadas para evitar tarifas mais altas.
> 
> **Fontes Relevantes:**
> 
> *   CNN Brasil
> *   Forbes
> *   Fundo Monetário Internacional (FMI)
> *   Estadão Verifica
> *   UFMG
> 


--------------------------------------------------------------
